# C14 DIstributed Variable and Accumulator 

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("c14").getOrCreate()
!netstat -anp |grep 4040 

tcp6       0      0 :::4040                 :::*                    LISTEN      4324/java           
tcp6       0      0 ::1:4040                ::1:50400               TIME_WAIT   -                   
tcp6       0      0 ::1:4040                ::1:50390               TIME_WAIT   -                   
tcp6       0      0 ::1:4040                ::1:50398               TIME_WAIT   -                   
tcp6       0      0 ::1:4040                ::1:50394               TIME_WAIT   -                   
tcp6       0      0 ::1:4040                ::1:50386               TIME_WAIT   -                   
tcp6       0      0 ::1:4040                ::1:50396               TIME_WAIT   -                   


In [2]:
# BROADCAST Variables 

supplement_data = {
"Spark" : 1000,
"Definitive" : 200,
"Big" : -300,
"Simple" : 100
  
}
type(supplement_data)

dict

In [3]:
suppBroadcast = spark.sparkContext.broadcast(supplement_data)

In [4]:
suppBroadcast.value

{'Spark': 1000, 'Definitive': 200, 'Big': -300, 'Simple': 100}

In [5]:
#words.map( lambda word: (word, suppBroadcast.value.get( word, 0)))\ .sortBy( lambda wordPair: wordPair[ 1])\ .collect()

my_collection = "Spark The Definitive Guide : Big Data Processing Made Simple"\
.split(" ") 

words = spark.sparkContext.parallelize( my_collection, 2)
words.collect()

words.map( lambda word: (word, suppBroadcast.value.get( word, 0)))\
.sortBy( lambda x: x[1])\
.collect()


[('Big', -300),
 ('The', 0),
 ('Guide', 0),
 (':', 0),
 ('Data', 0),
 ('Processing', 0),
 ('Made', 0),
 ('Simple', 100),
 ('Definitive', 200),
 ('Spark', 1000)]

In [6]:
# Accumulator Variable 

#suppBroadcast = spark.sparkContext.broadcast(supplement_data)
#Get the data  --> suppBroadcast.value 
var_acc = spark.sparkContext.accumulator(0)



In [14]:
#/root/golive/Spark-The-Definitive-Guide/data/flight-data/parquet/2010-summary.parquet
from pyspark.sql.functions import col
flights = spark.read.format("parquet")\
                    .option("path", "/root/golive/Spark-The-Definitive-Guide/data/flight-data/parquet/2010-summary.parquet")\
                    .load()

flights.show(2)
flights.select("*")\
.where(col("DEST_COUNTRY_NAME") == "China")\
.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|            China|      United States|  448|
+-----------------+-------------------+-----+



In [15]:
def check_china(flight_row):
    dest = flight_row["DEST_COUNTRY_NAME"]
    orig = flight_row["ORIGIN_COUNTRY_NAME"]
    if dest == "China":
        var_acc.add(flight_row["count"])
        
    if orig  == "China":
        var_acc.add(flight_row["count"])

In [17]:
flights.foreach(lambda flight_row: check_china(flight_row))

var_acc.value

953